# SYCL devices

In [275]:
%%writefile compute.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <vector>

int main() {
        // Example matrices
    int M = 2;
    int N = 2;
    int K = 2;

    std::vector<int> A(M * K), B(K * N), C(M * N);

//    for (int i = 0; i < M * K; ++i) A[i] = static_cast<float>(i % 10);
//    for (int i = 0; i < K * N; ++i) B[i] = static_cast<float>((i + 1) % 10);

    for (int i = 0; i < M * K; ++i) A[i] = i % 10;
    for (int i = 0; i < K * N; ++i) B[i] = (i + 1) % 10;


    auto myQueue = sycl::queue{sycl::cpu_selector_v};
    std::cout << "Running on "
              << myQueue.get_device().get_info<sycl::info::device::name>()
              << "\n";
    
    {
        auto bufA = sycl::buffer{A.data(), sycl::range<2>(M, K)};
        auto bufB = sycl::buffer{B.data(), sycl::range<2>(K, N)};
        auto bufC = sycl::buffer{C.data(), sycl::range<2>(M, N)};

        // Kernel to perform matrix-matrix multiplication
        myQueue.submit([&](sycl::handler& cgh) {
            sycl::accessor accA{bufA, cgh, sycl::read_only};
            sycl::accessor accB{bufB, cgh, sycl::read_only};
            sycl::accessor accC{bufC, cgh, sycl::write_only};

       //                                  _        _
       //                  _ __ ___   __ _| |_ _ __(_)_  __
       //                 | '_ ` _ \ / _` | __| '__| \ \/ /
       //                 | | | | | | (_| | |_| |  | |>  <
       //                 |_| |_| |_|\__,_|\__|_|  |_/_/\_\
       // 
       //                  _ _   _       _ _           _   _
       //  _ __ ___  _   _| | |_(_)_ __ | (_) ___ __ _| |_(_) ___  _ __
       // | '_ ` _ \| | | | | __| | '_ \| | |/ __/ _` | __| |/ _ \| '_ \
       // | | | | | | |_| | | |_| | |_) | | | (_| (_| | |_| | (_) | | | |
       // |_| |_| |_|\__,_|_|\__|_| .__/|_|_|\___\__,_|\__|_|\___/|_| |_|


        cgh.parallel_for(sycl::range<2>(M, N), [=](sycl::id<2> idx) {
            int row = idx[0];
            int col = idx[1];
            int sum = 0;
    
            for (int k = 0; k < K; ++k) {
                sum += accA[row][k] * accB[k][col];
                }

            accC[row][col] = sum;

            });
        });
        myQueue.wait();
    }

    std::cout << "Part of the resulting matrix C:" << std::endl;
    for (int i = 0; i < M; ++i) {
        for (int j = 0; j < N; ++j) {
            std::cout << C[i * N + j] << " ";
        }
        std::cout << std::endl;
    }

    return 0;
}

Overwriting compute.cpp


In [276]:
%%writefile ./run-sycl.sh
#!/bin/bash -x
source /opt/intel/oneapi/setvars.sh > /dev/null 2>&1
icpx -fsycl compute.cpp
if [ $? -eq 0 ]; then ./a.out; fi

Overwriting ./run-sycl.sh


In [277]:
!chmod u+x ./run-sycl.sh &&./run-sycl.sh

Running on 12th Gen Intel(R) Core(TM) i7-1280P
Part of the resulting matrix C:
3 4 
11 16 
